In [1]:
import os,sys, json,re, pickle
import magic, hashlib,  traceback ,ntpath, collections ,lief
from capstone import *
from capstone.x86 import *
import torch.nn as nn
import lief
from elftools.elf.elffile import ELFFile
from transformers import AdamW,AutoTokenizer
from tqdm import tqdm  # for our progress bar
from sklearn.metrics import precision_recall_fscore_support , accuracy_score,f1_score, confusion_matrix,mean_squared_error, mean_absolute_error, r2_score
from numpy import *
from num2words import num2words
import pandas as pd

In [2]:
ground_truth_json_dir = "/home/raisul/ANALYSED_DATA/ghidra_x86_pe_msvc_O2_static/"
#   
predict_json_dir_1 = "/home/raisul/ANALYSED_DATA/test/x86_pe_msvc_O2_static_1k_prob_disasm_4/"  #'/home/raisul/ANALYSED_DATA/prob_disasm_x86_pe_msvc_O2_static/'
predict_json_dir_2 = "/home/raisul/ANALYSED_DATA/test/x86_pe_msvc_O2_static_1k_prob_disasm/"
BIN_FILE_TYPE = "PE"
BIN_PATH = "/home/raisul/DATA/x86_pe_msvc_O2_static_stripped/"

In [3]:

def get_offsets(json_file_path):
    with open(json_file_path, "r") as file:
        data = json.load(file)
        temp = []
        for key,val in data.items():
            if 'int3' not in val.lower() and 'nop' not in val.lower():
                temp.append(int(key))
    return temp



def get_instructions(bin_file_path  ):
    md = Cs(CS_ARCH_X86, CS_MODE_64)
    md.detail = True
    offset_inst = {}

    with open(bin_file_path, 'rb') as f:
        try:
            if BIN_FILE_TYPE == "ELF":
                elffile = ELFFile(f)
                textSection = elffile.get_section_by_name('.text').data()
                text_section_offset = elffile.get_section_by_name('.text')['sh_offset']
              
            elif BIN_FILE_TYPE == "PE":
        
                        
                pe_file = lief.parse(bin_file_path)
                text_section = pe_file.get_section(".text")
                text_section_offset = text_section.pointerto_raw_data
                textSection = bytes(text_section.content)
                
        except Exception as e:
                print("An error occurred:", e ,bin_file_path)
                return
    
        for byte_index in range(len(textSection)):
            try:    
    
                instruction = next(md.disasm(textSection[byte_index: byte_index+15 ], text_section_offset + byte_index ), None)
                offset_inst[text_section_offset+byte_index] = instruction
                
                # if instruction:
                #     print("%d:\t%s\t%s _\t%x" %(int(instruction.address), instruction.mnemonic, instruction.op_str, instruction.size))
                # else:
                #     print("%d:\t%s " % (text_section_offset + byte_index  , 'invalid instruction') )
    
            except Exception as e:
                print(traceback.print_exc() )
                print(e)
    return offset_inst

ground_truth__jsons  =  os.listdir(ground_truth_json_dir) 
predict_jsons1    =  os.listdir(predict_json_dir_1)
predict_jsons2 = os.listdir(predict_json_dir_2)

common_json_files = list(set(ground_truth__jsons) & set(predict_jsons1) & set(predict_jsons2))
print(len(common_json_files))
common_json_files = common_json_files[0:10]
# common_json_files =['370cf902ac71f426c14725a24959e8de.json']
all_TP_1= all_FP_1= all_FN_1= all_TN_1 = 0
all_TP_2= all_FP_2= all_FN_2= all_TN_2 = 0

for json_file_name in common_json_files:
    # print('\n\n\n',json_file_name)
    ground_truth_json_path =  os.path.join(ground_truth_json_dir, json_file_name)
    predict_json_path_1 =  os.path.join(predict_json_dir_1, json_file_name)
    predict_json_path_2 =  os.path.join(predict_json_dir_2, json_file_name)
    
    ground_truth_offsets = get_offsets(ground_truth_json_path)
    predict_offsets_1 = get_offsets(predict_json_path_1)
    predict_offsets_2 = get_offsets(predict_json_path_2)
    
    ground_truth_offsets_set = set(ground_truth_offsets)
    predict_offsets_set_1 = set(predict_offsets_1)
    predict_offsets_set_2 = set(predict_offsets_2)

    
    superset = set(range( min(ground_truth_offsets) , max(ground_truth_offsets) ) )
    
    true_positive_1 = len(ground_truth_offsets_set&predict_offsets_set_1)
    false_positive_1 = len(predict_offsets_set_1 - ground_truth_offsets_set)
    false_negative_1 = len( ground_truth_offsets_set -predict_offsets_set_1 )
    true_negative_1 = len( superset - ( ground_truth_offsets_set|predict_offsets_set_1)  )

    true_positive_2 = len(ground_truth_offsets_set&predict_offsets_set_2)
    false_positive_2 = len(predict_offsets_set_2 - ground_truth_offsets_set)
    false_negative_2 = len( ground_truth_offsets_set -predict_offsets_set_2 )
    true_negative_2 = len( superset - ( ground_truth_offsets_set|predict_offsets_set_2)  )
    
    all_TP_1+=true_positive_1
    all_FP_1+=false_positive_1
    all_FN_1+=false_negative_1
    all_TN_1+=true_negative_1

    all_TP_2+=true_positive_2
    all_FP_2+=false_positive_2
    all_FN_2+=false_negative_2
    all_TN_2+=true_negative_2
    
    print('\ntrue_positive_1: ',true_positive_1 , '\n false_positive_1: ',false_positive_1 ,'\nfalse_negative_1:', false_negative_1 , '\ntrue_negative_1: ',true_negative_1)
    print('\ntrue_positive_2: ',true_positive_2 , '\n false_positive_2: ',false_positive_2 ,'\nfalse_negative_2:', false_negative_2 , '\ntrue_negative_2: ',true_negative_2)

    continue
    
    bin_file_path = os.path.join(os.path.join(BIN_PATH ,  json_file_name.split('.')[0] ),  (json_file_name.split('.')[0]+ ('.exe' if BIN_FILE_TYPE == "PE" else '' )))
    exhaustive_disasm = get_instructions(bin_file_path)
    # print(exhaustive_disasm)
    if exhaustive_disasm:
      for offset in exhaustive_disasm:
          if exhaustive_disasm[offset] is None:
              continue
          hash = '#' if offset in ground_truth_offsets else ' '
          dollar = '$' if offset in predict_offsets_set_1 else ' '
          star = '*' if offset in predict_offsets_set_2 else ' '
          marker = ''
          if offset in predict_offsets_set_2 and offset not in predict_offsets_set_1 :# 
              # 1 = isasm_4/"  
              marker = '     >>>>>>>>      '*3
          if hash=='#' or dollar=='$' or star=='*':
              print( hash , dollar ,star, hex(offset), exhaustive_disasm[offset].mnemonic+ exhaustive_disasm[offset].op_str ,' ', exhaustive_disasm[offset].size,marker) 
          

    else:
        print('exhaustive diasm none')
    # print(len(ground_truth_offsets) , len(predict_offsets))
    # false_negatives = list(set(ground_truth_offsets) - set(predict_offsets))
    



3

true_positive_1:  1512 
 false_positive_1:  19843 
false_negative_1: 116 
true_negative_1:  33

true_positive_2:  1628 
 false_positive_2:  133 
false_negative_2: 0 
true_negative_2:  15516

true_positive_1:  1640 
 false_positive_1:  20747 
false_negative_1: 114 
true_negative_1:  27

true_positive_2:  1754 
 false_positive_2:  277 
false_negative_2: 0 
true_negative_2:  16134

true_positive_1:  1325 
 false_positive_1:  19046 
false_negative_1: 87 
true_negative_1:  22

true_positive_2:  1412 
 false_positive_2:  229 
false_negative_2: 0 
true_negative_2:  14516


In [4]:
true_positive_1:  1748 
 false_positive_1:  277 
false_negative_1: 6 
true_negative_1:  16134

true_positive_2:  1754 
 false_positive_2:  277 
false_negative_2: 0 
true_negative_2:  16134

true_positive_1:  1409 
 false_positive_1:  229 
false_negative_1: 3 
true_negative_1:  14516

true_positive_2:  1412 
 false_positive_2:  229 
false_negative_2: 0 
true_negative_2:  14516

true_positive_1:  1625 
 false_positive_1:  133 
false_negative_1: 3 
true_negative_1:  15516

true_positive_2:  1628 
 false_positive_2:  133 
false_negative_2: 0 
true_negative_2:  15516


IndentationError: unexpected indent (3976145860.py, line 2)

In [ ]:
rue_positive_1:  1140 
 false_positive_1:  1674 
false_negative_1: 272 
true_negative_1:  12855

true_positive_2:  1412 
 false_positive_2:  229 
false_negative_2: 0 
true_negative_2:  14516

true_positive_1:  1313 
 false_positive_1:  2238 
false_negative_1: 315 
true_negative_1:  13291

true_positive_2:  1628 
 false_positive_2:  133 
false_negative_2: 0 
true_negative_2:  15516

true_positive_1:  1472 
 false_positive_1:  2305 
false_negative_1: 282 
true_negative_1:  13850

true_positive_2:  1754 
 false_positive_2:  277 
false_negative_2: 0 
true_negative_2:  16134

In [ ]:
279/ (279+ 1754) 


In [ ]:
how to truncate a float between 0 and 1
